In [14]:
import utils.functions as fun
from utils import dataset
from utils import nlp

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Parse input

In [15]:
tweets = dataset.read_tweets('DatasetFavCon')
tweets[:5]

[{'tweet_id': '5247',
  'category': 'AGAINST',
  'text': 'Victoria del sí, o victoria del NO?? Menos del 50% a favor en #eleccionescatalanas #27S'},
 {'tweet_id': '4103',
  'category': 'NEUTRAL',
  'text': 'El PP ja no té vot ocult a Catalunya. Sembla ser que Ciutadans ha capitalitzat el vot del no #27s Veurem què passa...'},
 {'tweet_id': '5468',
  'category': 'AGAINST',
  'text': '#27S Que la participación haya subido es buen dato. Que lo haga para desmantelar una mentira como la independentista sería lo ideal.'},
 {'tweet_id': '1797',
  'category': 'FAVOR',
  'text': '#Catalunya opta per #independència a les urnes. http://t.co/k4ysUuXRiq @laclaunoticies #JuntsPelSí #27S #HemGuanyat http://t.co/I4PVgbuKPa'},
 {'tweet_id': '4886',
  'category': 'AGAINST',
  'text': 'Los que plantean las elecciones del #27S como plebiscitarias son los que dicen que ganan con resultados parlamentarios. O lo uno o lo otro'}]

## Pre-processing dataset

In [16]:
pipeline = [
    nlp.remove_old_style_retweet_text,
    nlp.remove_hyperlinks,
    nlp.remove_hashtags,
    nlp.tokenize,
    nlp.reject_stopwords,
    nlp.reject_emoticons,
    nlp.reject_punctuations,
    nlp.stem
]

stem_sentences = [nlp.process_unit(pipeline, tweet['text']) for tweet in tweets]
stem_sentences[:2]

[['victoria', 'victoria', 'meno', '50', 'favor', 'eleccionescatalana', '27'],
 ['pp',
  'ja',
  'té',
  'vot',
  'ocult',
  'catalunya',
  'sembla',
  'ser',
  'ciutadan',
  'capitalitzat',
  'vot',
  '27',
  'veurem',
  'què',
  'passa',
  '...']]

In [17]:
result = fun.bag_of_words_for_svm(stem_sentences)
print(result['vocabulary_size'])

sentences = result['sentences']
sentences[0]

13902


array([0., 0., 0., ..., 0., 0., 0.])

In [18]:
labels = [tweet['category'] for tweet in tweets]
labels[:3]

['AGAINST', 'NEUTRAL', 'AGAINST']

## Predicting with a SVM

In [19]:
TEST_SIZE = 0.33
MAX_ITERATIONS = 500
X = sentences
y = labels

result = fun.classify_with_svm(X, y, TEST_SIZE, MAX_ITERATIONS)
result

/root/.local/share/virtualenvs/app-ueEJiAOq/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


{'total_correct': 1963, 'accuracy': 0.705861201006832}